In [1]:
# import numpy and pandas
import numpy as np
import pandas as pd

# used for dates
import datetime
from datetime import date

# Set some pandas options controlling output format
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 8)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 80)
pd.options.mode.chained_assignment = None

# bring in matplotlib for graphics
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
index1 = ['Max', 'Bithumb']
index2 = [1, 2, 3]
exchange_series = pd.Series(['Max','Max','Max', 'Max','Max','Max', 'Bithumb','Bithumb','Bithumb'])
position_series = pd.Series([1,1,1, 2,2,2, 3,3,3])
index3 = ['BCNT', 'USDT', 'TWD']
currency_series = pd.Series(np.tile(index3, len(index2)))

today_str = date.today().isoformat()
current_str = datetime.datetime.today().isoformat()
yesterday = date.today() - datetime.timedelta(days=1)
yesterday_date_str = yesterday.isoformat()

today_df = pd.DataFrame({
    'exchange': exchange_series,
    'position': position_series,
    'currency': currency_series,
    'date': np.repeat(current_str, len(index2)*len(index3)),
    'total': np.repeat(0.0, len(index2)*len(index3)),
    'locked': np.repeat(0.0, len(index2)*len(index3)),
}).set_index(['exchange','position','currency'])

In [13]:
today_df['total']['Bithumb',3,'BCNT'] = 1.0
today_df['total']['Bithumb',3,'USDT'] = 1.0
today_df['total']['Bithumb',3,'TWD'] = 1.0
today_df['total']['Max',1,'BCNT'] = 1.0
today_df['total']['Max',1,'USDT'] = 1.0
today_df['total']['Max',1,'TWD'] = 1.0
today_df['total']['Max',2,'BCNT'] = 1.0
today_df['total']['Max',2,'USDT'] = 1.0
today_df['total']['Max',2,'TWD'] = 1.0

today_df['locked']['Bithumb',3,'BCNT'] = 1.0
today_df['locked']['Bithumb',3,'USDT'] = 1.0
today_df['locked']['Bithumb',3,'TWD'] = 1.0
today_df['locked']['Max',1,'BCNT'] = 1.0
today_df['locked']['Max',1,'USDT'] = 1.0
today_df['locked']['Max',1,'TWD'] = 1.0
today_df['locked']['Max',2,'BCNT'] = 1.0
today_df['locked']['Max',2,'USDT'] = 1.0
today_df['locked']['Max',2,'TWD'] = 1.0

In [14]:
try:
  record_df = pd.read_csv('record.csv').set_index(['exchange','position','currency'])
  last_date = record_df[record_df.date < today_str].head(1).date
  if len(last_date.values) == 0:
    yesterday_df = today_df.copy()
  else:
    yesterday_df = record_df[record_df.date == last_date.values[0]][['total','locked']].copy()

  # pd.concat([record_df,today_df]).to_csv('record.csv')
except Exception as e:
  print(e)
  # read record file error => create new record file
  record_df = today_df.copy()
  yesterday_df = today_df.copy()

  # record_df.to_csv('record.csv')

[Errno 2] File record.csv does not exist: 'record.csv'


In [35]:
today_df = today_df[['total']]
max_today_df = pd.DataFrame({
    'exchange': ['Max','Max','Max'],
    'currency': ['BCNT','USDT','TWD'],
    'total': [
        today_df.xs('Max').xs('BCNT', level = 1).sum().values[0],
        today_df.xs('Max').xs('USDT', level = 1).sum().values[0],
        today_df.xs('Max').xs('TWD', level = 1).sum().values[0]
    ],
}).set_index(['exchange','currency'])
today_df = today_df.append(max_today_df)

yesterday_df = yesterday_df[['total']]
max_yesterday_df = pd.DataFrame({
    'exchange': ['Max','Max','Max'],
    'currency': ['BCNT','USDT','TWD'],
    'total': [
        yesterday_df.xs('Max').xs('BCNT', level = 1).sum().values[0],
        yesterday_df.xs('Max').xs('USDT', level = 1).sum().values[0],
        yesterday_df.xs('Max').xs('TWD', level = 1).sum().values[0]
    ],
}).set_index(['exchange','currency'])
yesterday_df = yesterday_df.append(max_yesterday_df)

In [37]:
diff_df = (today_df - yesterday_df)
diff_percentage = diff_df / yesterday_df
diff_df.columns = ['diff']
diff_df['diff_percentage'] = diff_percentage.total
diff_df[yesterday_date_str] = yesterday_df.total.apply(lambda x: round(x,2))
diff_df[today_str] = today_df.total.apply(lambda x: round(x,2))

diff_df['diff'] = diff_df['diff'].apply(lambda x: round(x,2))
diff_df['diff_percentage'] = diff_df['diff_percentage'].apply(lambda x: '{diff:>3.2f}%'.format(diff=round(x * 100,2)))

In [38]:
diff_df

                    diff diff_percentage  2020-04-15  2020-04-16
(Max, 1, BCNT)       0.0           0.00%         1.0         1.0
(Max, 1, USDT)       0.0           0.00%         1.0         1.0
(Max, 1, TWD)        0.0           0.00%         1.0         1.0
(Max, 2, BCNT)       0.0           0.00%         1.0         1.0
(Max, 2, USDT)       0.0           0.00%         1.0         1.0
...                  ...             ...         ...         ...
(Bithumb, 3, USDT)   0.0           0.00%         1.0         1.0
(Bithumb, 3, TWD)    0.0           0.00%         1.0         1.0
(Max, BCNT)          0.0           0.00%         2.0         2.0
(Max, USDT)          0.0           0.00%         2.0         2.0
(Max, TWD)           0.0           0.00%         2.0         2.0

[12 rows x 4 columns]